In [1]:
!pip freeze | grep scikit-learn

"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [33]:
!python -V

Python 3.9.18


In [34]:
import pickle
import pandas as pd

In [35]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [36]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [37]:
year = '2023'
month = '03'

In [38]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [16]:
y_pred.std()

6.247488852238703

In [21]:
df['ride_id'] = f'{int(year):04d}/{int(month):02d}_'+ df.index.astype('str')

In [28]:
df['prediction'] = y_pred

In [29]:
df_result = df[['ride_id', 'prediction']]

In [31]:
df_result.to_parquet(
    'output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [32]:
# !jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1397 bytes to starter.py
